EXERCISE 3

Imports : 

In [2]:
import cv2 as cv
import numpy as np

Taking as Input our Images :

In [3]:
noisyimage1 = cv.imread("noisyImage_SaltPepper_1.jpg",0)
print(noisyimage1)
print(noisyimage1.shape)

[[161 163 161 ... 170 156 126]
 [161 165 163 ... 168 155 129]
 [164 154 166 ... 172 154 129]
 ...
 [ 41  44  50 ... 103 102  98]
 [ 43  45  52 ... 106 106 106]
 [ 44  44  52 ... 103 103 109]]
(512, 512)


Necessary Function :

In [16]:
def medianFilter(image,medianFilterSize):
      m,n = image.shape
      paddingSize = medianFilterSize // 2
      new_matrix = borderReplicating(image,paddingSize)
      medianFilteredImage = np.zeros( (m,n) )
      for i in range(paddingSize,m+paddingSize):
            for j in range(paddingSize,n+paddingSize):
                  piece = take_a_piece(new_matrix,i,j,paddingSize)
                  piece = piece.reshape( (piece.shape[0]**2))
                  piece= np.sort(piece)
                  medianValue = piece[ (medianFilterSize**2) // 2 ]
                  medianFilteredImage[i-paddingSize,j-paddingSize] = medianValue
      median_filtered_image = np.uint8(np.round(medianFilteredImage) )
      return median_filtered_image

def centerWeightedmedianFilter(image,medianFilterSize,weight):
      m,n = image.shape
      paddingSize = medianFilterSize // 2
      new_matrix = borderReplicating(image,paddingSize)
      centerWeightedMedianFilteredImage = np.zeros( (m,n) )
      for i in range(paddingSize, paddingSize+m):
            for j in range(paddingSize, paddingSize+n):
                  piece = take_a_piece(new_matrix,i,j,paddingSize)
                  temp = applyingWeight(piece,paddingSize,weight)
                  temp = np.sort(temp)
                  medianValue = temp[ (medianFilterSize**2) // 2 ]
                  centerWeightedMedianFilteredImage[i-paddingSize, j-paddingSize]= medianValue
      centerWeightedMedianFilteredImage = np.uint8(np.round(centerWeightedMedianFilteredImage))
      return centerWeightedMedianFilteredImage

def applyingWeight(piece,paddingSize,weight):
      m,n = piece.shape
      listOfPieceValues = []
      for i in range(m):
            for j in range(n):
                  if i== paddingSize and j == paddingSize : # Which means it is center.
                        for k in range(weight):
                              listOfPieceValues.append(piece[i,j])
                  else:
                        listOfPieceValues.append(piece[i,j])
      return listOfPieceValues
            
def borderReplicating(noisyimage1,paddingSize):
      first_row = noisyimage1[0,:] 
      last_row = noisyimage1[-1,:]
      first_col = noisyimage1[:,0]
      last_col = noisyimage1[:,-1]
      m,n = noisyimage1.shape
      new_matrix= np.zeros( (m + 2 * paddingSize,n + 2 * paddingSize) )
      new_matrix[paddingSize : paddingSize + m,paddingSize : paddingSize + n] = noisyimage1
      for loop_range in range(paddingSize):
            new_matrix[loop_range, paddingSize : paddingSize + m] = first_row
            new_matrix[paddingSize + m + loop_range, paddingSize : paddingSize + m] = last_row
            new_matrix[paddingSize : paddingSize + n, loop_range] = first_col
            new_matrix[paddingSize: paddingSize + n ,paddingSize + n + loop_range  ] = last_col
      for i in range(paddingSize):
            for j in range(paddingSize):
                  new_matrix[i,j] = noisyimage1[0,0]
                  new_matrix[i, j+n+paddingSize] = noisyimage1[0,511]
                  new_matrix[i+paddingSize+m, j] = noisyimage1[511,0]
                  new_matrix[i+m+paddingSize,j+n+paddingSize] = noisyimage1[511,511]
      return new_matrix
def take_a_piece(big_matrix,i,j,padding_size):
      x1 = i-padding_size
      x2 = i+padding_size
      x3 = j-padding_size
      x4 = j+padding_size
      piece = big_matrix[x1:x2+1,x3:x4+1]
      return piece

def draw_images(image1, image2):
      cv.namedWindow("image1", cv.WINDOW_NORMAL)
      cv.namedWindow("image2", cv.WINDOW_NORMAL)
      cv.imshow("image1", image1)
      cv.imshow("image2", image2)
      cv.waitKey(0)
      cv.destroyAllWindows()   

Q-1 : Median Filter : 5x5

In [5]:
medianFilteredImage = medianFilter(noisyimage1,5)

OpenCV's Median Filter Implementation :

In [6]:
medianOutput = cv.medianBlur(noisyimage1, 5)


Evaluation Part :

Our Median Filter Implementation vs Noisy Image :

In [7]:
draw_images(medianFilteredImage,noisyimage1)

OpenCv's Median Fılter Implementation vs Noisy Image

In [8]:
draw_images(medianOutput,noisyimage1)

Our implementation vs OpenCV:

In [9]:
draw_images(medianFilteredImage,medianOutput)
print(np.sum(abs(medianFilteredImage-medianOutput)))

0


Q - 2 : PSNR's and Evaluation

In [10]:
originalImage = cv.imread("original.jpg",0)
print(originalImage)

[[163 162 161 ... 170 156 126]
 [162 162 161 ... 168 155 129]
 [162 162 162 ... 172 154 129]
 ...
 [ 41  44  50 ... 103 102  98]
 [ 43  45  52 ... 106 106 106]
 [ 44  44  52 ... 103 103 109]]


In [11]:
#DENOISE attempts:
# Our median filter.
medianfilterOutput = medianFilter(noisyimage1, 5)

# Opencv median filter.
medianOutput = cv.medianBlur(noisyimage1, 5)


# Opencv .blur() API - NORMALIZED box filter.
boxOutput = cv.blur(noisyimage1, (5,5))

# Opencv .GaussianBlur() API - Gaussian filter.
gaussOutput = cv.GaussianBlur(noisyimage1, (7,7), 0)



In [12]:
np.info(medianfilterOutput)

class:  ndarray
shape:  (512, 512)
strides:  (512, 1)
itemsize:  1
aligned:  True
contiguous:  True
fortran:  False
data pointer: 0x1feb90efea0
byteorder:  little
byteswap:  False
type: uint8


PSNR's : Peak Signal Noise Ratio's

In [30]:
def psnr_vals(originalImage, noisyimage1, boxOutput, gaussOutput, medianOutput,medianfilterOutput):
    psnr = cv.PSNR(originalImage, noisyimage1)
    psnrBOX = cv.PSNR(originalImage, boxOutput)
    psnrGAUSS = cv.PSNR(originalImage, gaussOutput)
    psnrMEDIAN = cv.PSNR(originalImage, medianOutput)
    psnrMEDIAN2 = cv.PSNR(originalImage, medianfilterOutput)
    print(psnr)
    print(psnrBOX)
    print(psnrGAUSS)
    print(psnrMEDIAN)
    print(psnrMEDIAN2)
psnr_vals(originalImage,noisyimage1,boxOutput,gaussOutput,medianOutput,medianfilterOutput)

28.362689687778783
29.2751171865568
30.177481427164818
31.195753661063716
31.195753661063716


We can clearly see that median filters are better in the area of denoising salt and pepper noises. PSNR is a metric which measures how good a denoised image matches with the original image and we can see above the psnr values are increasing while going  "box -> gaussian -> median" and that is because actually our noise is all 0's and 255's , so when the median filter is sorting the area of filter_size array, these values are going to edges of array, and finally we can use the median value. But on the other hand, gaussian and box filter use these pixel values(noised ones).
So, according to our logic and PSNR values on the above, median filters are better in the case of SALT AND PEPPER noises.

Q - 3 : Center Weighted Median Filter : ( The weights are : 1 1 1 1 1 / 1 1 3 1 1 / 1 1 1 1 1 ) Center weight is 3.

In [18]:
# Inputs: Noisy Image, Median Filter Size, Center Pixels Value.
centerMedianFilterOutput = centerWeightedmedianFilter(noisyimage1,5,3)
draw_images(centerMedianFilterOutput,noisyimage1)

[[161 161 161 ... 169 156 129]
 [161 161 161 ... 168 155 129]
 [161 161 161 ... 169 154 129]
 ...
 [ 44  45  50 ... 100 100  98]
 [ 44  45  49 ... 103 103 103]
 [ 44  44  51 ... 103 103 106]]


In [20]:
psnrMEDIANCENTERWEIGHTED = cv.PSNR(originalImage,centerMedianFilterOutput)
print(psnrMEDIANCENTERWEIGHTED)

31.989662722239437


PSNR Values Revisited : 

In [41]:
psnr_vals(originalImage,noisyimage1,boxOutput,gaussOutput,medianOutput,medianfilterOutput)
print("Center Weighted Median Filter : " ,psnrMEDIANCENTERWEIGHTED)

28.362689687778783
29.2751171865568
30.177481427164818
31.195753661063716
31.195753661063716
Center Weighted Median Filter :  31.989662722239437


Q-4 : Applying Filters to More and More Images 

In [23]:
noisyImage2 = cv.imread("noisyImage_SaltPepper_2.jpg",0)
noisyImage3 = cv.imread("noisyImage_SaltPepper_3.jpg",0)
noisyImage4 = cv.imread("noisyImage_SaltPepper_4.jpg",0)

Applying Median Filter to Images :

In [24]:
medianFilterOutput2=medianFilter(noisyImage2,5)
medianFilterOutput3=medianFilter(noisyImage3,5)
medianFilterOutput4=medianFilter(noisyImage4,5)


medianOutput2 = cv.medianBlur(noisyImage2, 5)
medianOutput3 = cv.medianBlur(noisyImage3, 5)
medianOutput4 = cv.medianBlur(noisyImage4, 5)

centerMedianFilterOutput2 = centerWeightedmedianFilter(noisyImage2,5,3)
centerMedianFilterOutput3 = centerWeightedmedianFilter(noisyImage3,5,3)
centerMedianFilterOutput4 = centerWeightedmedianFilter(noisyImage4,5,3)


Applying Box Filter to Images :

In [36]:
boxOutput2 = cv.blur(noisyImage2, (5,5))
boxOutput3 = cv.blur(noisyImage3, (5,5))
boxOutput4 = cv.blur(noisyImage4, (5,5))

Applying Gaussian Filter to Images :

In [37]:
gaussOutput2 = cv.GaussianBlur(noisyImage2, (7,7), 0)
gaussOutput3 = cv.GaussianBlur(noisyImage3, (7,7), 0)
gaussOutput4 = cv.GaussianBlur(noisyImage4, (7,7), 0)

PSNR Values of Other Images : 

In [38]:
psnr_vals(originalImage,noisyImage2,boxOutput2,gaussOutput2,medianOutput2,medianFilterOutput2)
psnrMEDIANCENTERWEIGHTED2 = cv.PSNR(originalImage,centerMedianFilterOutput2)
print(psnrMEDIANCENTERWEIGHTED2)

25.252754200220195
29.0182996640224
29.858708656756274
31.181800680424118
31.181800680424118
31.95582877629777


In [39]:
psnr_vals(originalImage,noisyImage3,boxOutput3,gaussOutput3,medianOutput3,medianFilterOutput3)
psnrMEDIANCENTERWEIGHTED3 = cv.PSNR(originalImage,centerMedianFilterOutput3)
print(psnrMEDIANCENTERWEIGHTED3)

22.338767790966788
28.533705033633545
29.274268615123017
31.10931114360617
31.10931114360617
31.867775626953986


In [40]:
psnr_vals(originalImage,noisyImage4,boxOutput4,gaussOutput4,medianOutput4,medianFilterOutput4)
psnrMEDIANCENTERWEIGHTED4 = cv.PSNR(originalImage,centerMedianFilterOutput4)
print(psnrMEDIANCENTERWEIGHTED4)

19.414920543390515
27.6796220470695
28.260512479802532
31.029963635969523
31.029963635969523
31.781615946259386
